In [1]:
import sqlite3
import pandas as pd
from tqdm import tqdm
import os
from dotenv import load_dotenv

from langchain_community.utilities import SQLDatabase
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

/nmhs2/hari/work/UAV/Projects/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Connect to SQLite database (or create it if it doesn't exist)
connection = sqlite3.connect("student_grades.db")
cursor = connection.cursor()

# Create a table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS grades (
        id INTEGER PRIMARY KEY,
        name TEXT,
        subject TEXT,
        score INTEGER,
        grade TEXT
    )
""")

# Insert some dummy data
data = [
    (1, "Aman", "Math", 95, "A"),
    (2, "Anshu", "Math", 78, "C"),
    (3, "Akshu", "History", 88, "B"),
    (4, "Rahul", "History", 92, "A"),
    (5, "Divyansh", "Science", 85, "B"),
    (6, "Nandini", "Math", 65, "D")
]

cursor.executemany("INSERT OR IGNORE INTO grades VALUES (?, ?, ?, ?, ?)", data)
connection.commit()
connection.close()

print("Database created and populated successfully!")

Database created and populated successfully!


In [3]:
db = SQLDatabase.from_uri("sqlite:///student_grades.db")

In [4]:
schema = db.get_table_info()
print("📋 Database schema:")
print(schema)

📋 Database schema:

CREATE TABLE grades (
	id INTEGER, 
	name TEXT, 
	subject TEXT, 
	score INTEGER, 
	grade TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from grades table:
id	name	subject	score	grade
1	Aman	Math	95	A
2	Anshu	Math	78	C
3	Akshu	History	88	B
*/


In [5]:
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")


llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0.1
)

# Step 5: Define the SQL generation prompt
prompt = ChatPromptTemplate.from_template("""
You are a senior data analyst and SQL expert.

Given the database schema below, write a correct SQL query
that answers the user's question.

Rules:
- Use only the tables and columns in the schema
- Do NOT explain anything
- Return ONLY the SQL query

Schema:
{schema}

Question:
{question}
""")

# Step 6: Create the LangChain pipeline
sql_chain = (
    prompt
    | llm
    | StrOutputParser()
)

In [6]:
# Example input
question = input("Enter your question: ")

if question:
    try:
        # Generate SQL query using the chain
        sql_query = sql_chain.invoke(
            {"schema": schema, "question": question}
        ).strip()

        print("\n🧠 Generated SQL:")
        print(sql_query)

        # Execute query
        print("\n📈 Result:")
        result = db.run(sql_query)
        print(result)

    except Exception as e:
        print(f"\n❌ Error: {e}")



🧠 Generated SQL:
SELECT name
FROM grades
WHERE score = (SELECT MAX(score) FROM grades);

📈 Result:
[('Aman',)]
